In [50]:

%%HTML
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [51]:
# CELL 1
import SPOCK.stats as SPOCKstats
import SPOCK.long_term_scheduler as SPOCKLT
import SPOCK.short_term_scheduler as SPOCKST
import SPOCK.plots_scheduler as SPOCKplot
from astropy.time import Time
import pandas as pd
from astropy.utils import iers
import matplotlib.pyplot as plt
iers.IERS_A_URL  ='ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all'
import yaml
import sys
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML, display, clear_output, Javascript
from plotly import graph_objs as go
from IPython.display import Image
import os
from astropy import units as u
import datetime
from astroplan import Observer
%matplotlib notebook
from astropy.coordinates import SkyCoord,EarthLocation,CartesianRepresentation
import markdown 
from ipywidgets import HTML

from ipywidgets import TwoByTwoLayout
import warnings
from ipywidgets import GridspecLayout
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 2000
pd.options.display.max_columns = 80

#from astroplan import download_IERS_A
#download_IERS_A

#check_user = widgets.Text(description='Username',style=style)
#check_password = widgets.Password(description='Password',style=style)

#credentials = widgets.HBox([check_user,check_password],layout=widgets.Layout(height='40px'))
#display(credentials)

# Utils widgets


In [52]:
# CELL 2

style = {'description_width':'initial'}

observatory_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose site',
    options=['SSO', 'SNO', 'Saint-Ex', 'TS_La_Silla','TN_Oukaimeden'],
    description='Site:',
    ensure_option=True,
    disabled=False)

telescope_widget = widgets.Combobox(
    # value='John',
    placeholder='SSO telescope',
    options=['Io','Europa','Ganymede','Callisto'],
    description='Telescope:',
    ensure_option=True,
    disabled=False)

operator_widget = widgets.Combobox(
    # value='John',
    placeholder='Opertor for TS/TN',
    options=['Manu','Youssef','Fran','Khalid','Marin'],
    description='Operator:',
    ensure_option=True,
    disabled=False)

start_date_widget =widgets.DatePicker(
    description='Start date',
    disabled=False)

end_date_widget =widgets.DatePicker(
    description='End date',
    disabled=False)

input_file_widget = widgets.Text(description='File',placeholder='input.csv')

run_widget = widgets.Button(
    value=False,
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')

short_term_option_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose option',
    options=['Follow-up','Special target with known start/end time',\
             'Special target (observe the most we can)','Dome rotation','Monitoring'],
    description='What kind? :',
    ensure_option=True,
    disabled=False)

short_day_widget =widgets.DatePicker(
    description='Day ',
    disabled=False)

short_start_time_widget = widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00')
short_end_time_widget = widgets.Text(description='End time',placeholder='2020-01-17 03:00:00')

start_time_widget =widgets.DatePicker(
    description='Start Time',
    disabled=False)

end_time_widget =widgets.DatePicker(
    description='End Time',
    disabled=False)

input_file_special_widget = widgets.Text(description='File',placeholder='input_short_term.csv')
 
input_target_list_widget = widgets.Text(description='File',placeholder='./target_lists/speculoos_target_list_v6.txt')
    
target_widget = widgets.Combobox(
    # value='John',
    placeholder='Spxxx-xxxx',
    options=list(pd.read_csv('./target_lists/speculoos_target_list_v6.txt',delimiter=' ')['Sp_ID']), 
    description='Target name:',
    ensure_option=True,
    disabled=False)

target_special_widget = widgets.Combobox(
    # value='John',
    placeholder='Trappist-1x',
    options=list(pd.read_csv('./target_lists/target_list_special.txt',delimiter=' ')['Sp_ID']), 
    description='Target special:',
    ensure_option=True,
    disabled=False)

target_follow_up_widget = widgets.Combobox(
    # value='John',
    placeholder='Trappist-1x',
    options=list(pd.read_csv('./target_lists/target_transit_follow_up.txt',delimiter=' ')['Sp_ID']), 
    description='Target follow-up:',
    ensure_option=True,
    disabled=False)

plot_option_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose a plot option',
    options=['Gant chart','Visibility  plot','Phase coverage','Global gant chart'],
    description='What kind? :',
    ensure_option=True,
    disabled=False)

#check_a_target_widget = widgets.Button(value=False,description='Select a target',disabled=False,
#        button_style='info',tooltip='show target list',icon='fa-arrow-right')

check_np_widget = widgets.Button(
    value=False,
    description='See night plans',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to see night plans',
    icon='check')

info_target_widget = widgets.Button(
    value=False,
    description='Info on a target',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to chose target',
    icon='check')

def run_all(b): 
    display(Javascript('Jupyter.notebook.execute_cells([3])'))


# ----------------------------------------------------------- #
    
long_term_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget,operator_widget]),\
                                        widgets.HBox([start_date_widget,end_date_widget]), \
                                        input_file_widget,\
                                       run_widget])
long_term_tab.set_title(0, 'Observatory')
long_term_tab.set_title(1, 'Date range')
long_term_tab.set_title(2, 'Input file')
long_term_tab.set_title(3, 'Run long term')

# ----------------------------------------------------------- #

short_term_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget,operator_widget]),\
                                        input_file_special_widget,widgets.VBox([widgets.HBox([short_term_option_widget,short_day_widget]),\
                                        widgets.HBox([short_start_time_widget,short_end_time_widget]),
                                             widgets.HBox([target_special_widget,target_follow_up_widget])]),
                                             run_widget])
short_term_tab.set_title(0, 'Observatory')
short_term_tab.set_title(1, 'Input file')
short_term_tab.set_title(2, 'Option')
short_term_tab.set_title(3, 'Run short term')

# ----------------------------------------------------------- #

plot_tab = widgets.Accordion(children=[input_file_widget,plot_option_widget,input_target_list_widget,run_widget])
plot_tab.set_title(0, 'Input file')
plot_tab.set_title(1, 'Plot option')
plot_tab.set_title(2, 'Target list')
plot_tab.set_title(3, 'Run plots')

# ----------------------------------------------------------- #

make_upload_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget]),\
                                          widgets.HBox([start_date_widget,end_date_widget]),\
                                          run_widget])
make_upload_tab.set_title(0, 'Site info')
make_upload_tab.set_title(1, 'Date info')
make_upload_tab.set_title(2, 'Go to plan maker and upload')

# ----------------------------------------------------------- #

check_np_tab = widgets.Accordion(children=[check_np_widget])
make_upload_tab.set_title(0, 'Click')

# ----------------------------------------------------------- #

info_target_tab = widgets.Accordion(children=[info_target_widget])
make_upload_tab.set_title(0, 'Click')

# ----------------------------------------------------------- #

ra_text= widgets.FloatText(
    value=346.620833,
    description='RA:',
    disabled=False
)
dec_text =  widgets.FloatText(
    value=-5.0411111,
    description='DEC:',
    disabled=False
)
timing_text =  widgets.FloatText(
    value=2457322.51505,
    description='Timing (BJD):',
    disabled=False
)
period_text =  widgets.FloatText(
    value=1.51088212,
    description='Period (days):',
    disabled=False
)
duration_text =  widgets.FloatText(
    value=0.02511108,
    description='Duration (days):',
    disabled=False
)
name_widget = widgets.Text(description='Target name',placeholder='TOI-XXX')
prediction_date_widget =widgets.DatePicker(
    description='Start date',
    disabled=False)

nb_transits_widget = widgets.IntText(
    value=4,
    description='Nb transits:',
    disabled=False
)

coordinates = TwoByTwoLayout(top_left=ra_text, top_right=dec_text,
                     bottom_left=prediction_date_widget, bottom_right=name_widget)

ephemerids = TwoByTwoLayout(top_left=timing_text, top_right=period_text ,
                     bottom_left=duration_text, bottom_right=nb_transits_widget)

predict_tab = widgets.Accordion(children=[coordinates,ephemerids,run_widget ])

predict_tab.set_title(0, 'Coordinates')
predict_tab.set_title(1, 'Ephemerids')
predict_tab.set_title(2, 'Run')



# ----------------------------------------------------------- #

run_widget.on_click(run_all)
check_np_widget.on_click(run_all)
info_target_widget.on_click(run_all)

tab_nest = widgets.Tab()
tab_nest.children = [long_term_tab, short_term_tab,plot_tab,make_upload_tab,check_np_tab,info_target_tab, predict_tab]
tab_nest.set_title(0, 'Long term scheduler ')
tab_nest.set_title(1, 'Short term scheduler')
tab_nest.set_title(2, 'Plots')
tab_nest.set_title(3, 'Save & Upload only')
tab_nest.set_title(4, 'See night plans')
tab_nest.set_title(5, 'Info on a target')
tab_nest.set_title(6, 'Predictions')
tab_nest

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
# CELL 3

if tab_nest.selected_index == 0:
    if input_file_widget.value is '':
        input_file = 'input.csv'
    else:
        input_file = input_file_widget.value
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    else:
        if observatory_widget.value == 'SSO':
            obs = 1
        if observatory_widget.value == 'SNO':
            obs = 2
        if observatory_widget.value == 'Saint-Ex':
            obs = 3
        if observatory_widget.value == 'TS_La_Silla':
            obs = 4
        if observatory_widget.value == 'TN_Oukaimeden':
            obs = 5
    if start_date_widget.value is '' and end_date_widget.value is '':
        sys.exit('ERROR: The start and/or end date are not valid')

    start_date = Time(datetime.datetime.strptime(str(start_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
    end_date = Time(datetime.datetime.strptime(str(end_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
    schedule = SPOCKLT.Schedules()
    schedule.load_parameters(input_file,obs)
    schedule.date_range = Time([start_date,end_date])
    if observatory_widget.value == 'SSO':
        schedule.telescope = str(telescope_widget.value)
    if schedule.telescope == 'Saint-Ex':
        schedule.make_schedule(Altitude_constraint = 28, Moon_constraint = 30)
        display(schedule.target_table_spc[schedule.idx_first_target])
        display(schedule.target_table_spc[schedule.idx_second_target])
        display(Javascript('Jupyter.notebook.execute_cells([6])'))
    else:
        schedule.make_schedule(Altitude_constraint = 25, Moon_constraint = 30)
        display(schedule.target_table_spc[schedule.idx_first_target])
        display(schedule.target_table_spc[schedule.idx_second_target])
        display(Javascript('Jupyter.notebook.execute_cells([6])'))

if tab_nest.selected_index == 1:
    if input_file_widget.value is '':
        input_file = 'input_short_term.csv'
    else:
        input_file = input_file_widget.value
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    else:
        if observatory_widget.value == 'SSO':
            obs = 1
        if observatory_widget.value == 'SNO':
            obs = 2
        if observatory_widget.value == 'Saint-Ex':
            obs = 3
        if observatory_widget.value == 'TS_La_Silla':
            obs = 4
        if observatory_widget.value == 'TN_Oukaimeden':
            obs = 5

    schedule = SPOCKST.Schedules()
    schedule.load_parameters(input_file,obs)
    if observatory_widget.value == 'SSO':
        schedule.telescope = str(telescope_widget.value)
    schedule.use = short_term_option_widget.value
    day = Time(datetime.datetime.combine(short_day_widget.value,datetime.time(15,0,0)),format='datetime').iso
    #schedule.day_of_night = Time(schedule.day_of_night.iso)
    schedule.day_of_night = Time(day)
    
    if short_term_option_widget.value == 'Follow-up':
        if target_follow_up_widget.value != '':
            schedule.transit_follow_up('./target_lists/target_transit_follow_up.txt',name=target_follow_up_widget.value)
        else:
            schedule.transit_follow_up('./target_lists/target_transit_follow_up.txt')
    if short_term_option_widget.value == 'Special target with known start/end time':
        start_date = [Time(short_start_time_widget.value).iso]
        end_date = [Time(short_end_time_widget.value).iso]
        schedule.start_end_range = Time([start_date,end_date])
        input_name = target_special_widget.value
        schedule.special_target_with_start_end(input_name)
    if short_term_option_widget.value == 'Special target (observe the most we can)':
        input_name = target_special_widget.value
        schedule.special_target(input_name)
    if short_term_option_widget.value == 'Dome rotation':
        schedule.dome_rotation()
    if short_term_option_widget.value == 'Monitoring':
        start_date = Time(short_start_time_widget.value).iso
        end_date = Time(short_end_time_widget.value).iso
        schedule.start_end_range = Time([start_date,end_date])
        input_name = target_special_widget.value
        schedule.monitoring(input_name,5,61)
    schedule.make_scheduled_table()
    schedule.planification()
    schedule.make_night_block()
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
    SPOCKST.make_plans(day=schedule.day_of_night,nb_days=1,telescope=schedule.telescope)
    display(schedule.scheduled_table_sorted)
    display(Javascript('Jupyter.notebook.execute_cells([6])'))
    
if tab_nest.selected_index == 2:   
    if plot_option_widget.value == 'Visibility  plot':
        display(Javascript('Jupyter.notebook.execute_cells([4])'))
        
    if plot_option_widget.value == 'Phase coverage':
        display(Javascript('Jupyter.notebook.execute_cells([10])'))

    if plot_option_widget.value == 'Global gant chart':
        schedule = SPOCKLT.Schedules()
        schedule.load_parameters('input.csv',1)
        SPOCKplot.gantt_chart_all(schedule.target_list)
        
    if plot_option_widget.value == 'Gant chart':
        display(Javascript("Jupyter.notebook.execute_cells([5])"))

if tab_nest.selected_index == 3:
    if input_file_widget.value is '':
        input_file = 'input.csv'
    else:
        input_file = input_file_widget.value
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    else:
        if observatory_widget.value == 'SSO':
            obs = 1
        if observatory_widget.value == 'SNO':
            obs = 2
        if observatory_widget.value == 'Saint-Ex':
            obs = 3
        if observatory_widget.value == 'TS_La_Silla':
            obs = 4
        if observatory_widget.value == 'TN_Oukaimeden':
            obs = 5
    if start_date_widget.value is '' and end_date_widget.value is '':
        sys.exit('ERROR: The start and/or end date are not valid')
    else:
        start_date = Time(datetime.datetime.strptime(str(start_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
        end_date = Time(datetime.datetime.strptime(str(end_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
        schedule = SPOCKLT.Schedules()
        schedule.load_parameters(input_file,obs)
        schedule.date_range = Time([start_date,end_date])
        if observatory_widget.value == 'SSO':
            schedule.telescope = str(telescope_widget.value)
    display(Javascript("Jupyter.notebook.execute_cells([6])"))

if tab_nest.selected_index == 4:
    display(Javascript('Jupyter.notebook.execute_cells([11])'))
    
if tab_nest.selected_index == 5:
    display(Javascript('Jupyter.notebook.execute_cells([12])'))
    
if tab_nest.selected_index == 6:
    display(Javascript('Jupyter.notebook.execute_cells([13])'))

target,start time (UTC),end time (UTC),duration (minutes),ra (h),ra (m),ra (s),dec (d),dec (m),dec (s),configuration
str11,str23,str23,float64,float64,float64,float64,float64,float64,float64,object
NGTS-11,2020-12-03 18:25:00.008,2020-12-04 00:32:20.008,367.3333333333334,1.0,34.0,5.148000000001645,-14.0,-25.0,-8.93639999999678,"{'texp=10', 'filt=g'}"
Sp0627+5651,2020-12-04 00:32:20.008,2020-12-04 06:18:20.020,346.00020000000006,6.0,27.0,38.22664800000652,56.0,51.0,40.470120000002794,"{'texp=68', 'filt=I+z'}"


<IPython.core.display.Javascript object>

In [ ]:
# CELL 4
@interact
def plt_target(Site=['Saint-Ex','SSO','SNO','TS_La_Silla','TN_Oukaimeden'],Target=list(pd.read_csv('./target_lists/speculoos_target_list_v6.txt',delimiter=' ')['Sp_ID']),day_widget = widgets.DatePicker(description='Pick a Date')):
    if Site== 'SSO':
        obs = 1
    if Site== 'SNO':
        obs = 2
    if Site== 'Saint-Ex':
        obs = 3
    if Site== 'TS_La_Silla':
        obs = 4
    if Site== 'TN_Oukaimeden':
        obs = 5
    schedule = SPOCKLT.Schedules()
    schedule.load_parameters(input_file,obs)
    try:
        day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0)),format='datetime')
        SPOCKplot.airmass_altitude_plot_given_target(name_observatory=Site,day=day,target=Target,path_target_list='./target_lists/speculoos_target_list_v6.txt')

    except TypeError:
        print('ERROR: Choose a date ! ')

In [ ]:
# CELL 5
@interact
def plt_gant_chart(Telescope=['Europa','Io','Callisto','Ganymede','Artemis','Saint-Ex','TS_La_Silla','TN_Oukaimeden','all'],start_widget = widgets.DatePicker(description='Start Date'),end_widget = widgets.DatePicker(description='End Date')):
    try:
        day = Time(datetime.datetime.combine(start_widget,datetime.time(15,0,0)),format='datetime')
        if Telescope == 'all':
            Telescope = ['Europa','Io','Callisto','Ganymede','Artemis','Saint-Ex','TS_La_Silla','TN_Oukaimeden']
        SPOCKplot.gantt_chart(day,day+1,Telescope)
    except (TypeError,NameError) as e:
        print('ERROR: Choose date range ! '),

In [56]:
# CELL 6
save_widget = widgets.Button(
    value=False,
    description='Save plans',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save',
    icon='check'
)
def run_save(b): 
    display(Javascript('Jupyter.notebook.execute_cells([7])'))

display(save_widget)

save_widget.on_click(run_save)


Button(description='Save plans', icon='check', style=ButtonStyle(), tooltip='Save')

<IPython.core.display.Javascript object>

In [57]:
# CELL 7
if tab_nest.selected_index == 0:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.save_schedule(None,obs,True,True,schedule.date_range,schedule.telescope)
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        SPOCKLT.upload_plans(start_date, nb_days=schedule.date_range_in_days,telescope = schedule.telescope)

    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.save_schedule(None,obs,True,True,schedule.date_range,schedule.telescope)
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        SPOCKLT.upload_plans(start_date, nb_days=schedule.date_range_in_days,telescope = schedule.telescope)
        
    if schedule.observatory.name == 'SSO':
        SPOCKLT.save_schedule(None,obs,True,True,schedule.date_range,schedule.telescope)
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        SPOCKLT.upload_plans(start_date, nb_days=schedule.date_range_in_days,telescope = schedule.telescope)

    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKLT.save_schedule(None,obs,True,True,schedule.date_range,schedule.telescope)
        display(Javascript("Jupyter.notebook.execute_cells([8])"))
        #SPOCKLT.make_docx_schedule(schedule.observatory,schedule.telescope,schedule.date_range,operator_widget.value,schedule.target_list)
        
    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKLT.save_schedule(None,obs,True,True,schedule.date_range,schedule.telescope)
        display(Javascript("Jupyter.notebook.execute_cells([8])"))
        #SPOCKLT.make_docx_schedule(schedule.observatory,schedule.telescope,schedule.date_range,operator_widget.value,schedule.target_list)

if tab_nest.selected_index == 1:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKST.save_schedule(None,obs,True,True,schedule.day_of_night,schedule.telescope)
        SPOCKST.make_np(schedule.day_of_night,1,schedule.telescope)
        SPOCKST.upload_plans(schedule.day_of_night, nb_days=1,telescope = schedule.telescope)
        
    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKST.save_schedule(None,obs,True,True,schedule.day_of_night,schedule.telescope)
        SPOCKST.make_np(schedule.day_of_night,1,schedule.telescope)
        SPOCKST.upload_plans(schedule.day_of_night, nb_days=1,telescope = schedule.telescope)
        
    if schedule.observatory.name == 'SSO':
        SPOCKST.save_schedule(None,obs,True,True,schedule.day_of_night,schedule.telescope)
        SPOCKST.make_np(schedule.day_of_night,1,schedule.telescope)
        SPOCKST.upload_plans(schedule.day_of_night, nb_days=1,telescope = schedule.telescope)
         
    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKST.save_schedule(None,obs,True,True,schedule.day_of_night,schedule.telescope)
        display(Javascript("Jupyter.notebook.execute_cells([8])"))
        
    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKST.save_schedule(None,obs,True,True,schedule.day_of_night,schedule.telescope)
        display(Javascript("Jupyter.notebook.execute_cells([8])"))
        
if tab_nest.selected_index == 3:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        SPOCKLT.upload_plans(start_date, nb_days=schedule.date_range_in_days,telescope = schedule.telescope)

    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        SPOCKLT.upload_plans(start_date, nb_days=schedule.date_range_in_days,telescope = schedule.telescope)
        
    if schedule.observatory.name == 'SSO':
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        SPOCKLT.upload_plans(start_date, nb_days=schedule.date_range_in_days,telescope = schedule.telescope)

    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKLT.make_docx_schedule(schedule.observatory,schedule.observatory.name,schedule.date_range,operator_widget.value)
        SPOCKLT.upload_plans(start_date, nb_days=schedule.date_range_in_days,telescope = schedule.telescope)
        
    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKLT.make_docx_schedule(schedule.observatory,schedule.observatory.name,schedule.date_range,operator_widget.value)
        SPOCKLT.upload_plans(start_date, nb_days=schedule.date_range_in_days,telescope = schedule.telescope)


<IPython.core.display.Javascript object>

In [58]:
# Cell 8

docx_widget = widgets.Button(
    value=False,
    description='Update .docx file',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Docx',
    icon='check'
)
start_date_docx_widget =widgets.DatePicker(
    description='Start .docx',
    disabled=False)

end_date_docx_widget =widgets.DatePicker(
    description='End .docx',
    disabled=False)

def run_save(b): 
    display(Javascript('Jupyter.notebook.execute_cells([9])'))
    
    
docx = widgets.Accordion(children=[widgets.HBox([start_date_docx_widget,end_date_docx_widget,docx_widget])])

display(docx)

docx_widget.on_click(run_save)



Accordion(children=(HBox(children=(DatePicker(value=None, description='Start .docx'), DatePicker(value=None, d…

<IPython.core.display.Javascript object>

In [59]:
# Cell 9
schedule = SPOCKLT.Schedules()
schedule.observatory = observatory_widget.value
schedule.telescope = observatory_widget.value
start_date = Time(datetime.datetime.strptime(str(start_date_docx_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
end_date = Time(datetime.datetime.strptime(str(end_date_docx_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
schedule.date_range = Time([start_date,end_date])
SPOCKLT.make_docx_schedule(SPOCKLT.charge_observatories(schedule.observatory)[0],schedule.telescope,schedule.date_range,operator_widget.value)



KeyError: -1

In [61]:
SPOCKLT.charge_observatories(schedule.observatory)[0],schedule.telescope,schedule.date_range,operator_widget.value

(<Observer: name='TN_Oukaimeden',
     location (lon, lat, el)=(-7.8622630000000004 deg, 31.205160000000003 deg, 2750.999999999263 m),
     timezone=<UTC>>,
 'TN_Oukaimeden',
 <Time object: scale='utc' format='iso' value=['2020-12-02 15:00:00.000' '2020-12-10 15:00:00.000']>,
 '')

In [ ]:
# Cell 10 
@interact
def plt_phase_cov(Site=['SSO','SNO','Saint-Ex','TS_La_Silla','TN_Oukaimeden'],Target=list(pd.read_csv('./target_lists/speculoos_target_list_v6.txt',delimiter=' ')['Sp_ID']),periods = widgets.FloatRangeSlider(
    value=[0, 6],
    min=0,
    max=20.0,
    step=0.1,
    description='Period:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',)):
    
    if Site== 'SSO':
        obs = 1
    if Site== 'SNO':
        obs = 2
    if Site== 'Saint-Ex':
        obs = 3
    if Site== 'TS_La_Silla':
        obs = 4
    if Site== 'TN_Oukaimeden':
        obs = 5
    schedule = SPOCKLT.Schedules()
    schedule.load_parameters(input_file,obs)

    SPOCKplot.phase_coverage_given_target(name_observatory=Site,target=Target,path_target_list='./target_lists/speculoos_target_list_v6.txt',pmin=periods[0],pmax=periods[1])
    

In [ ]:
# Cell 11 check night  plans
@interact
def night_block(telescope=['Saint-Ex','Io','Europa','Ganymede','Callisto','Artemis','TS_La_Silla','TN_Oukaimeden'],day_widget = widgets.DatePicker(description='Pick a Date')):
    try:
        day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0))).iso
        date = Time(day,out_subfmt='date').iso
        nightp = SPOCKstats.read_night_plans_server(telescope,date)
        if nightp.columns[0] == '<!DOCTYPE':
            print('ERROR: No night plan on this date ! ')
        else: 
            display(nightp)
    except TypeError:
        print('ERROR: Choose a date ! ')


In [ ]:
# Cell 12 Info targets
@interact
def plt_target(Target=list(pd.read_csv('./target_lists/speculoos_target_list_v6.txt',delimiter=' ')['Sp_ID'])):
    target_list_v6 = pd.read_csv('./target_lists/speculoos_target_list_v6.txt',sep=' ')
    a = target_list_v6.where((target_list_v6['Sp_ID']==Target))
    a  = a.dropna()
    display(a)
    #try:
    #    day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0)),format='datetime')
    #    SPOCKstats.read_night_plans_server(telescope,date)
#
    #except TypeError:
    #    print('ERROR: Choose a date ! ')

In [ ]:
# Cell 13
day = Time(datetime.datetime.combine(prediction_date_widget.value,datetime.time(15,0,0)),format='datetime').iso
SPOCKST.prediction(obs_name='SSO',name=name_widget.value,ra=ra_text.value,dec=dec_text.value,
                   timing=timing_text.value,
                   period=period_text.value,duration=duration_text.value,
                   start_date=day,ntr=nb_transits_widget.value)